In [26]:
# A scientific computing package
import numpy as np

# Data visualization & analysis package
import pandas as pd

# Data preprocessing
# Deep-learning package but we only use this package for
# some preprocessing tasks
import tensorflow as tf
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer

# Sklearn provides some machine-learning function
# TFIDF for convert documents to vector representation
# Euclidean Distance tell us the distance between vectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import euclidean_distances

# Utilities
from langdetect import detect
from googletrans import Translator

In [4]:
data = pd.read_csv("./data/books.csv",index_col=0)

In [5]:
data.sample(20)

,Id,Title,description
452,492,Collins English For Life - Writing (A2 - Pre I...,NaN
161,623,The Digital Quality Handbook: A Guide for Achi...,As mobile and web technologies continue to exp...
144,327,The Last Lecture,"A lot of professors give talks titled ""The Las..."
14,367,"Global Outsourcing: Executing an Onshore, Near...",Global outsourcing is defined as the seamless ...
465,525,Oxford Learner's Pocket Phrasal Verbs and Idioms,•Learn phrases and expressions for everyday si...
395,141,Cẩm Nang Kinh Doanh - Giao Tiếp Thương Mại,Bộ sách nổi tiếng Cẩm nang kinh doanh Harvard ...
33,217,Payment Technologies for E-Commerce,Electronic payment is the economic backbone of...
408,165,Cẩm Nang Kinh Doanh - Kỹ Năng Thương Lượng,Bộ sách nổi tiếng “Cẩm nang Kinh doanh Harvard...
46,422,Carrots and Sticks Don't Work: Build a Culture...,Advance praise for Carrots and Sticks Don't Wo...
187,387,How to Reduce the Cost of Software Testing,Plenty of software testing books tell you how ...


## Corpus Cleaning

### Remove books that have no description

Because we depend on the description of books to measure the similarity between them, a book without a description cannot be used

In [6]:
no_des = data[data['description'].isnull()]
print(f"Number of books without a description {len(no_des)}")
no_des.head(10)

Number of books without a description 54


,Id,Title,description
396,151,Cẩm Nang Kinh Doanh - Quản Lý Khủng Hoảng,NaN
412,169,Đắc Nhân Tâm,NaN
413,170,Sắp Xếp Quyền Ưu Tiên Cho Công Việc,NaN
418,196,Toàn Cảnh Công Nghiệp Phần Mềm Việt Nam 2010,NaN
419,213,The Big Book of Humorous Training Games,NaN
420,222,Phát triển kỹ năng Lãnh Đạo,NaN
421,223,QBQ - Tư duy thông minh,NaN
422,274,Collective Intelligence in Action,NaN
423,290,Pocket Glossary of Health Information Manageme...,NaN
424,308,HL7 - CDA: an XML example UH-201,NaN


In [7]:
# Drop books that have no description
data = data[~data['description'].isnull()].reset_index(drop=True)

### Translate vietnamese description to english
It's not a good idea to work with 2 languages simultaneously in a NLP problem, so we will do our best to convert all vietnamese description to english with Google Translate Api, it's not very accurate but it will do for now.

In [8]:
not_english = []
for idx,row in data.iterrows():
    des = row['description']
    lang = detect(des)
    if (lang!='en'):
        not_english.append(row['Id'])
not_english = data[data['Id'].isin(not_english)]

In [9]:
print(f"Number of books with a description that not in 'English': {len(not_english)}")
not_english.head(10)

Number of books with a description that not in 'English': 62


,Id,Title,description
360,101,Bí Quyết Thành Công - Để Hiệu Quả Trong Công Việc,"Kỹ năng xác định những mục tiêu ưu tiên, khắc ..."
361,102,Sức Mạnh Của Sự Khích Lệ (Bí Quyết Thành Công ...,Thông qua câu chuyện thú vị có thật về việc hu...
362,103,Bí Quyết Tuyển Dụng Và Đãi Ngộ Người Tài,"""Bí Quyết Tuyển Dụng Và Đãi Ngộ Người Tài"" là ..."
363,104,Bí Quyết Thành Công - Câu Chuyện Nhà Quản Lý C...,Bạn đang cầm trên tay quyển sách đã được dịch ...
364,105,Cẩm Nang Quản Lý Hiệu Quả - Quản Lý Sự Thay Đổi,Cuốn sách Quản Lý Sự Thay Đổi này sẽ hướng dẫn...
365,106,Cẩm Nang Quản Lý Hiệu Quả - Tư Duy Chiến Lược,Có khả năng hoạch định dài hạn trong khi vẫn p...
366,107,Cẩm Nang Quản Lý Hiệu Quả - Quản Lý Toàn Cầu,Những kỹ năng thực tế hướng dẫn bạn cách thích...
367,108,Cẩm Nang Quản Lý Hiệu Quả - Quản Lý Ngân Sách,Cách sử dụng và quản lý ngân sách như một công...
368,109,Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Phỏng Vấn,"Việc phỏng vấn tuyển dụng nhân viên, dù là thư..."
369,110,Cẩm Nang Quản Lý Hiệu Quả - Hành Động Hiệu Quả,"""Người bình thường chờ đợi và thực hiện tuần t..."


In [10]:
trans = Translator()

In [11]:
for idx,row in not_english.iterrows():
    translations = trans.translate(row['description']).text
    not_english.at[idx,'description'] = translations
    data.at[idx,'description'] = translations

In [12]:
not_english.sample(10)

,Id,Title,description
420,534,Academic Essay Writing - Tự Học Viết Luận,ACADEMIC WRITING Essay Book (SELF STUDY essay)...
413,175,7 Thói Quen Để Thành Đạt,"Increasingly complex life, stress and harsh as..."
368,109,Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Phỏng Vấn,"The recruitment interview, whether regularly o..."
409,167,Hạt Giống Tâm Hồn - Thay Thái Độ Đổi Cuộc Đời ...,Attitude is Everything by Keith D. Harrell for...
423,555,Sẵn sàng cho mọi việc,You do not need to work harder. Even you may n...
402,159,Cẩm Nang Kinh Doanh - Quản Lý Dự Án Lớn Và Nhỏ,Ministry of Harvard Business Plan - Harvard Bu...
411,172,Trò Chơi Tiên Đoán - Dự Báo Và Định Trước Tươn...,Bruce Bueno de Mesquita is a master of game th...
367,108,Cẩm Nang Quản Lý Hiệu Quả - Quản Lý Ngân Sách,Usage and budget management as a business tool...
400,157,Cẩm Nang Kinh Doanh - Xây Dựng Nhóm Làm Việc H...,Ministry of Harvard Business Plan - Harvard Bu...
403,160,Cẩm Nang Kinh Doanh - Quản Lý Tính Sáng Tạo & ...,The book provides the basic guidelines for a m...


### Turn table format to array

In [13]:
corpus = data['description'].values

In [14]:
corpus

array(['Companies of all sizes are finding an urgent new priority--keeping employees. This book takes the reader through the critical ideas of employee retention using what is called the Three Rs of Employee Retention--Respect, Recognition, and Rewards--as the basis for any program designed to keep people around.',
       "Written by a software developer for software developers, this book is a unique collection of the latest software development methods. The author includes OOD, UML, Design Patterns, Agile and XP methods with a detailed description of a complete software design for reusable programs in C++ and Java. Using a practical, problem-solving approach, it shows how to develop an object-oriented application—from the early stages of analysis, through the low-level design and into the implementation. Walks readers through the designer's thoughts — showing the errors, blind alleys, and creative insights that occur throughout the software design process. The book covers: Statics and

## Data preprocessing

In [15]:
string = "hey you, I say hello, so hello to you."
vocab = string.split(' ')
vocab

['hey', 'you,', 'I', 'say', 'hello,', 'so', 'hello', 'to', 'you.']

### Filter out noise symbols
Because of the way we tokenize our corpus (using white space as a seperator), noise symbols like " , . ' ( ) [ ] { } : ; - can really mess-up our vocab library because
    
    hello != hello, != "hello != hello.

In [16]:
token = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='!"#$%&()*,-./:;<=>?@[\\]^_`{|}~\t\n',)
token.fit_on_texts(corpus)
corpus = token.sequences_to_texts(token.texts_to_sequences(corpus))

In [17]:
corpus[:2]

['companies of all sizes are finding an urgent new priority keeping employees this book takes the reader through the critical ideas of employee retention using what is called the three rs of employee retention respect recognition and rewards as the basis for any program designed to keep people around',
 "written by a software developer for software developers this book is a unique collection of the latest software development methods the author includes ood uml design patterns agile and xp methods with a detailed description of a complete software design for reusable programs in c++ and java using a practical problem solving approach it shows how to develop an object oriented application—from the early stages of analysis through the low level design and into the implementation walks readers through the designer's thoughts — showing the errors blind alleys and creative insights that occur throughout the software design process the book covers statics and dynamics principles of class des

### Filter out stopwords
Stopwords are words that consider to be unimportant in a document because they are just too common.

Example of stopwords include: *__is, am, are, the, which, what, how, at, on, has, have, etc...__*

In [18]:
stop_words = stopwords.words('english')

In [19]:
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [20]:
for idx, description in enumerate(corpus):
    tokenized_description = word_tokenize(description)
    new_description = [w for w in tokenized_description if not w in stop_words]
    corpus[idx] = " ".join(new_description)

In [21]:
corpus

['companies sizes finding urgent new priority keeping employees book takes reader critical ideas employee retention using called three rs employee retention respect recognition rewards basis program designed keep people around',
 "written software developer software developers book unique collection latest software development methods author includes ood uml design patterns agile xp methods detailed description complete software design reusable programs c++ java using practical problem solving approach shows develop object oriented application—from early stages analysis low level design implementation walks readers designer 's thoughts — showing errors blind alleys creative insights occur throughout software design process book covers statics dynamics principles class design complexity management principles package design analysis design patterns paradigm crossings explains principles ood one one demonstrates numerous examples completely worked designs case studies covers traps pitfall

### Stemming
Stemming is the process of converting a word to its basic form. The idea is that these morphological variants only use for grammatically purpose instead of having any extra meaning. 

For example:

    looks -> look

    looking -> look

    looked -> look

In [22]:
ps = PorterStemmer()

In [23]:
sentence = "Programers program with programing languages"
words = word_tokenize(sentence) 
for w in words: 
    print(w, " : ", ps.stem(w)) 

Programers  :  program
program  :  program
with  :  with
programing  :  program
languages  :  languag


In [24]:
for idx, description in enumerate(corpus):
    tokenized_description = word_tokenize(description)
    new_description = [ps.stem(w) for w in tokenized_description]
    corpus[idx] = " ".join(new_description)

In [25]:
corpus

['compani size find urgent new prioriti keep employe book take reader critic idea employe retent use call three rs employe retent respect recognit reward basi program design keep peopl around',
 "written softwar develop softwar develop book uniqu collect latest softwar develop method author includ ood uml design pattern agil xp method detail descript complet softwar design reusabl program c++ java use practic problem solv approach show develop object orient application—from earli stage analysi low level design implement walk reader design 's thought — show error blind alley creativ insight occur throughout softwar design process book cover static dynam principl class design complex manag principl packag design analysi design pattern paradigm cross explain principl ood one one demonstr numer exampl complet work design case studi cover trap pitfal work around applic c++ ood show agil method use discuss method design develop big softwar detail featur three chapter depth singl case studi b

## TFIDF
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

**TFIDF**, short for "*term frequency–inverse document frequency*", is a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.[1] It is often used as a weighting factor in searches of information retrieval, text mining, and user modeling. TFIDF is one of the most popular term-weighting schemes today; 83% of text-based recommender systems in digital libraries use TFIDF.

**TFIDF** equation include 2 parts, **TF** and **IDF**

### TF ( Term Frequency )
TF = how often a word appear in a doccument

$TF(i,j)=\frac{\text{Word i frequency in document j}}{\text{Number of words in document j}}$

### IDF (Inversed Document Frequency)
IDF = how many documents contain a particular word. This make sure common words doesn't take too much weight when compare documents

$IDF(i) = \log{(\frac{Number of documents}{Number of documents the word i appear in})}$

### Final Equation
$TFIDF = TF\cdot{IDF}$ 

![title](assets/tfidf-sample.png)

In [27]:
tfidf = TfidfVectorizer()
doc_vectors = tfidf.fit_transform(corpus)

In [28]:
print("Shape of our document matrix",doc_vectors.shape)

Shape of our document matrix (440, 5589)


## Build RS systems

In content-based RS, we recommend based on the **similarity** between **new items** with the **items users has interacted with**.   

### How to measure similarity

There are many metrics we can use when to try to measure the similarity between 2 vectors:

- cosine similarity
- pearson similarity
- euclidean distance
- etc

### Euclidean Distance

![title](./assets/Euclidean_distance.png)

To measure the distance between point $(x1,y1)$ and $(x2,y2)$ we can use the Pythagorean Theorem:

$$d = \sqrt{(x_1-x_2)^2 + (y_1-y_2)^2}$$

However, the Pythagorean Theorem doesn't limit us to 2-dimensional space. Infact, this theorem work at any N dimensions. The general equation to calculate the distance between 2 point __*P*__ and __*Q*__ in N-dimension:

$$d = \sqrt{(P_1-Q_1)^2+(P_2-Q_2)^2 + ... + (P_n-Q_n)^2}$$
$$\text{Where:}$$
$$P = <P_1,P_2,...,P_n>$$
$$Q = <Q_1,Q_2,...,Q_n>$$

In [30]:
def similar_books(bookIndex,k=10,timeit=False):
    target = doc_vectors[bookIndex]    
    dist = [euclidean_distances(target,i)[0][0] for i in doc_vectors]
    dist = np.argsort(dist)
    recommended = data.iloc[dist][1:k+1]['Title']
    if not timeit:
        print(f"Similar books to '{data.iloc[bookIndex]['Title']}'\n")
        for b in recommended:
            print(b)

In [33]:
similar_books(231)

Similar books to 'CBAP / CCBA Exam Prep, Premier Edition: A Course in a Book for Becoming an IIBA Certified Business Analyst'

Business Analysis: Best Practices for Success
AWS Certified Solutions Architect - Professional: The Definitive Study Guide
Cẩm Nang Kinh Doanh - Tài Chính Dành Cho Người Quản Lý
Head First Pmp: A Brain-Friendly Guide to Passing the Project Management Professional Exam
Oxford Business English Dictionary for learners of English Dictionary and CD-ROM Pack
A Guide to the Business Analysis Body of Knowledge (BABOK Guide)
HL7 Study Guide
Market Leader Intermediate Coursebook and DVD-Rom Pack 3rd Edition
PMP Exam Prep, Seventh Edition
Cẩm Nang Kinh Doanh - Huấn Luyện Và Truyền Kinh Nghiệm


In [34]:
doc_vectors

<440x5589 sparse matrix of type '<class 'numpy.float64'>'
	with 35159 stored elements in Compressed Sparse Row format>

In [35]:
row=440
col=5589
size=8 #bytes
print(f"Total memory ~{np.prod([row,col,size])/1e6} mb")

Total memory ~19.67328 mb


In [36]:
%timeit similar_books(21,10,True)

166 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Pre-compute the similarity matrix

In [37]:
books_similarity = euclidean_distances(doc_vectors)

In [38]:
books_similarity.shape

(440, 440)

In [39]:
row=440
col=440
size=8 #bytes
print(f"Total memory ~ {np.prod([row,col,size])/1e6} mb")

Total memory ~ 1.5488 mb


In [40]:
def similar_books_v2(bookIndex,k=10,timeit=False):
    dist = np.argsort(books_similarity[bookIndex])
    recommended = data.iloc[dist][1:k+1]['Title']
    if not timeit:
        print(f"Similar books to '{data.iloc[bookIndex]['Title']}'\n")
        for b in recommended:
            print(b)

In [41]:
similar_books_v2(21)

Similar books to 'Interviewer Success: Become a Great Interviewer in Less Than One Hour'

Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Phỏng Vấn
"Headhunter" Hiring Secrets: The Rules of the Hiring Game Have Changed . . . Forever!
Hiring the Best: Manager's Guide to Effective Interviewing and Recruiting, Fifth Edition
Growing Great Employees: Turning Ordinary People into Extraordinary Performers
Strengths-Based Leadership
Cẩm Nang Quản Lý Hiệu Quả - Thông Tin Hiệu Quả
Bí Quyết Tuyển Dụng Và Đãi Ngộ Người Tài
The Business Analyst's Handbook
Fun Works: Creating Places Where People Love to Work
Cẩm Nang Quản Lý Hiệu Quả - Kinh Doanh Hiệu Quả


In [42]:
%timeit similar_books_v2(21,10,True)

498 µs ± 3.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [43]:
160 / (599 / 1000)

267.1118530884808

### Recommend new books to users

![title](./assets/becauseyouwatch.png)

In [44]:
transaction = pd.read_csv("./data/transactions.csv")

In [45]:
transaction

,Title,Action,Check Out To,Request Status,Borrow From,Borrow To,Transaction Created,Handle Books Copies and Action,Created By,Item Type,Path
0,Request: Accelerate: The Science of Lean Softw...,Check out,Giang Thi Tra Nguyen,Requested,4/2/2019,5/2/2019,2.0,5.0,Giang Thi Tra Nguyen,Item,library/Lists/Transactions
1,Request: Continuous Delivery: Reliable Softwar...,Check out,Uyen Nguyen,Requested,4/1/2019,5/1/2019,2.0,5.0,Uyen Nguyen,Item,library/Lists/Transactions
2,Request: AWS Certified Developer - Associate G...,Check out,Nhan Quach,Requested,4/1/2019,5/1/2019,2.0,5.0,Nhan Quach,Item,library/Lists/Transactions
3,Request: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng T...,Check out,Thong Huynh,Requested,3/29/2019,4/23/2019,2.0,5.0,Thong Huynh,Item,library/Lists/Transactions
4,Request: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng T...,Check out,Thong Huynh,Requested,3/29/2019,4/23/2019,2.0,5.0,Thong Huynh,Item,library/Lists/Transactions
5,Request: Host Your Web Site In The Cloud: Amaz...,Check in,Thao Thi Phuong Nguyen (Izenda),Checked Out,3/28/2019,4/28/2019,2.0,5.0,Thao Thi Phuong Nguyen (Izenda),Item,library/Lists/Transactions
6,"Request: Agile Software Development, Principle...",View,Thao Thi Phuong Nguyen (Izenda),Checked In,3/5/2019,4/27/2019,2.0,5.0,Thao Thi Phuong Nguyen (Izenda),Item,library/Lists/Transactions
7,Request: Big Data For Dummies,View,Nghia Trong Nguyen,Checked In,3/6/2019,4/27/2019,2.0,5.0,Nghia Trong Nguyen,Item,library/Lists/Transactions
8,Request: Effective JavaScript: 68 Specific Way...,View,Tu Anh Nguyen,Checked In,1/16/2019,3/27/2019,5.0,5.0,Tu Anh Nguyen,Item,library/Lists/Transactions
9,Request: How to Reduce the Cost of Software Te...,View,Thao Thi Phuong Nguyen (Izenda),Checked In,3/6/2019,3/27/2019,2.0,5.0,Thao Thi Phuong Nguyen (Izenda),Item,library/Lists/Transactions


In [46]:
transaction = transaction[['Title','Check Out To']].rename(columns={'Title':'title','Check Out To':'user'})

In [47]:
transaction

,title,user
0,Request: Accelerate: The Science of Lean Softw...,Giang Thi Tra Nguyen
1,Request: Continuous Delivery: Reliable Softwar...,Uyen Nguyen
2,Request: AWS Certified Developer - Associate G...,Nhan Quach
3,Request: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng T...,Thong Huynh
4,Request: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng T...,Thong Huynh
5,Request: Host Your Web Site In The Cloud: Amaz...,Thao Thi Phuong Nguyen (Izenda)
6,"Request: Agile Software Development, Principle...",Thao Thi Phuong Nguyen (Izenda)
7,Request: Big Data For Dummies,Nghia Trong Nguyen
8,Request: Effective JavaScript: 68 Specific Way...,Tu Anh Nguyen
9,Request: How to Reduce the Cost of Software Te...,Thao Thi Phuong Nguyen (Izenda)


In [48]:
for i,x in transaction.iterrows():
    title = x['title']
    transaction.at[i,'title'] = title[9:]

In [49]:
transaction

,title,user
0,Accelerate: The Science of Lean Software and D...,Giang Thi Tra Nguyen
1,Continuous Delivery: Reliable Software Release...,Uyen Nguyen
2,AWS Certified Developer - Associate Guide: You...,Nhan Quach
3,Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Thương Lượng,Thong Huynh
4,Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Thuyết Trình,Thong Huynh
5,Host Your Web Site In The Cloud: Amazon Web Se...,Thao Thi Phuong Nguyen (Izenda)
6,"Agile Software Development, Principles, Patter...",Thao Thi Phuong Nguyen (Izenda)
7,Big Data For Dummies,Nghia Trong Nguyen
8,Effective JavaScript: 68 Specific Ways to Harn...,Tu Anh Nguyen
9,How to Reduce the Cost of Software Testing,Thao Thi Phuong Nguyen (Izenda)


In [50]:
def book_read_by_user(user):
    books = transaction[transaction['user']==user]['title']
    books = data[data['Title'].isin(books)]['Title'].values
    for b in books:
        print('-',b)

In [51]:
book_read_by_user('Dung Vo')

- The Web Application Hacker's Handbook: Finding and Exploiting Security Flaws
- Behind the Cloud: The Untold Story of How Salesforce.com Went from Idea to Billion-Dollar Company-and Revolutionized an Industry
- Accelerate: The Science of Lean Software and DevOps: Building and Scaling High Performing Technology Organizations
- Domain-Driven Design: Tackling Complexity in the Heart of Software
- Don't Make Me Think, Revisited: A Common Sense Approach to Web Usability (3rd Edition)
- Working Effectively with Legacy Code


In [52]:
def recommend_to_user(user,k=5):
    books = transaction[transaction['user']==user]['title']
    books_idx = data[data['Title'].isin(books)].index.values
    for idx in books_idx:
        dist = np.argsort(books_similarity[idx])
        recommened = data.iloc[dist][:k+1]['Title'].values
        print("Because you read:",recommened[0])
        print()
        for b in recommened[1:]:
            print("\t",b)
        print("--------------------------------")

In [55]:
recommend_to_user("Thong Huynh")

Because you read: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Thuyết Trình

	 Cẩm Nang Quản Lý Hiệu Quả - Hành Động Hiệu Quả
	 Cẩm Nang Quản Lý Hiệu Quả - Thông Tin Hiệu Quả
	 Cẩm Nang Quản Lý Hiệu Quả - Phân Công Hiệu Quả
	 Cẩm Nang Quản Lý Hiệu Quả - Đánh Giá Năng Lực Nhân Viên
	 Power Through Presentations: Tips and Tricks to Build a Better Slide Deck
--------------------------------
Because you read: Cẩm Nang Quản Lý Hiệu Quả - Kỹ Năng Thương Lượng

	 Cẩm Nang Kinh Doanh - Kỹ Năng Thương Lượng 
	 Getting To Yes: Negotiating Agreement Without Giving In
	 Cẩm Nang Quản Lý Hiệu Quả - Đánh Giá Năng Lực Nhân Viên
	 Cẩm Nang Quản Lý Hiệu Quả - Thông Tin Hiệu Quả
	 Cẩm Nang Quản Lý Hiệu Quả - Quản Lý Toàn Cầu
--------------------------------
Because you read: Big Data - Tuyển tập dữ liệu lớn

	 Big Data For Dummies
	 Big Data: Principles and best practices of scalable realtime data systems
	 Hadoop in Practice
	 Agile Data Science 2.0: Building Full-Stack Data Analytics Applications with Spark
	 